In [ ]:
# import necessary libraries
import pandas as pd 
import datetime
from zipfile import ZipFile
import glob
import zipfile
import os
import pyarrow as pa
import pyarrow.parquet as pq
import shutil
import requests

In [14]:
# function for latest date
def latest_date(file_list):
    date_list = []
    for file_name in file_list:
        date_ = datetime.datetime.strptime(file_name.split('_')[6], '%Y%m%d').date()
        date_list.append(date_)

    max_date = max(date_list)
    return max_date

list_files = [file[10:] for file in glob.glob('zip_files/*.zip')]
start_date = latest_date(list_files) + datetime.timedelta(1)
end_date = datetime.datetime.now().date()

df = pd.read_excel('./nse_data_url_temp.xlsx')
date_format = "%Y%m%d"

# Function to generate URL based on the date
def generate_url(current_date,type = ""):
  if type=='bhavcopy':
    url = f"https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_{current_date}_F_0000.csv.zip"
  else:
    url = f"https://nsearchives.nseindia.com/archives/equities/mkt/MA{current_date}.csv"
  return url



# Function to generate URLs from start date till today, excluding weekends
def dates_till_now(current_date,df,today=datetime.datetime.now().date()):
    urls = []
    # Loop through the dates from start date to today
    isUpdate = False
    while current_date <= today:
        # Exclude weekends (Saturday and Sunday)
        if current_date.weekday() != 5 and current_date.weekday() != 6:
            # Generate URL
            urlForBhav = generate_url(current_date.strftime(date_format),"bhavcopy")
            urlForMkt =  generate_url(current_date.strftime("%d%m%y"))
            print(urlForBhav)
            print(urlForMkt)
          

            new_row_df = pd.DataFrame([{
               "BhavCopy": urlForBhav,"MarketReport":urlForMkt,"date":current_date
            }])


            df = pd.concat([df, new_row_df], ignore_index=True)
            isUpdate = True
            
            
            # Append URL to the list
            urls.append({"BhavCopy": urlForBhav,"MarketReport":urlForMkt,"date":current_date})
        # Move to the next day
        current_date += timedelta(days=1)
    if isUpdate:
      df.to_excel('./nse_data_url_temp.xlsx', index=False)
      
dates_till_now(start_date,df,end_date)



https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240729_F_0000.csv.zip
https://nsearchives.nseindia.com/archives/equities/mkt/MA290724.csv
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240730_F_0000.csv.zip
https://nsearchives.nseindia.com/archives/equities/mkt/MA300724.csv
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240731_F_0000.csv.zip
https://nsearchives.nseindia.com/archives/equities/mkt/MA310724.csv
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240801_F_0000.csv.zip
https://nsearchives.nseindia.com/archives/equities/mkt/MA010824.csv
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240802_F_0000.csv.zip
https://nsearchives.nseindia.com/archives/equities/mkt/MA020824.csv
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240805_F_0000.csv.zip
https://nsearchives.nseindia.com/archives/equities/mkt/MA050824.csv
https://nsearchives.nseindia.com/content/cm/BhavCopy

In [15]:


def moveFiles(base_dir,target_dir):
    for filename in os.listdir(base_dir):
        basePath = os.path.join(base_dir,filename)
        targetPath = os.path.join(target_dir,filename)
        shutil.move(basePath,targetPath)

def extract_all_zips(directory):
    """
    Extract all zip files in the specified directory.
    
    Parameters:
    directory (str): The path to the directory containing zip files.
    """
    # Ensure the directory exists
    if not os.path.isdir(directory):
        print(f"The directory {directory} does not exist.")
        return

    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".zip"):
            zip_path = os.path.join(directory, filename)
            extract_path = os.path.join(directory, os.path.splitext(filename)[0])
            finalPath =  os.path.join(directory, os.path.splitext(filename)[0],os.path.splitext(filename)[0])

            # Create a directory for the extracted files
            if not os.path.exists(extract_path):
                os.makedirs(extract_path)

            # Extract the zip file
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_path)
                print(f"Extracted {filename} to {extract_path}")



            # Define file paths
            existing_parquet_file = './NSE-Data-2024.parquet'

            # Read the existing Parquet file
            existing_df = pd.read_parquet(existing_parquet_file)

            # Read new records from a CSV file
            new_df = pd.read_csv(finalPath)

            # Concatenate existing DataFrame with new DataFrame
            updated_df = pd.concat([existing_df, new_df], ignore_index=True)

            # Convert the updated DataFrame to a PyArrow Table
            table = pa.Table.from_pandas(updated_df)

            # Write the table to the Parquet file (overwrite or append)
            pq.write_table(table, existing_parquet_file)

            print(f"Updated dataset saved to {existing_parquet_file}")

# Example usage:
# Specify the directory containing the zip files
directory = "./zip_files_downloaded"
extract_all_zips(directory)
moveFiles('./zip_files_downloaded','./zip_files')


Extracted BhavCopy_NSE_CM_0_0_0_20240729_F_0000.csv.zip to ./zip_files_downloaded\BhavCopy_NSE_CM_0_0_0_20240729_F_0000.csv
Updated dataset saved to ./NSE-Data-2024.parquet
Extracted BhavCopy_NSE_CM_0_0_0_20240730_F_0000.csv.zip to ./zip_files_downloaded\BhavCopy_NSE_CM_0_0_0_20240730_F_0000.csv
Updated dataset saved to ./NSE-Data-2024.parquet
Extracted BhavCopy_NSE_CM_0_0_0_20240731_F_0000.csv.zip to ./zip_files_downloaded\BhavCopy_NSE_CM_0_0_0_20240731_F_0000.csv
Updated dataset saved to ./NSE-Data-2024.parquet
Extracted BhavCopy_NSE_CM_0_0_0_20240801_F_0000.csv.zip to ./zip_files_downloaded\BhavCopy_NSE_CM_0_0_0_20240801_F_0000.csv
Updated dataset saved to ./NSE-Data-2024.parquet
Extracted BhavCopy_NSE_CM_0_0_0_20240802_F_0000.csv.zip to ./zip_files_downloaded\BhavCopy_NSE_CM_0_0_0_20240802_F_0000.csv
Updated dataset saved to ./NSE-Data-2024.parquet
Extracted BhavCopy_NSE_CM_0_0_0_20240805_F_0000.csv.zip to ./zip_files_downloaded\BhavCopy_NSE_CM_0_0_0_20240805_F_0000.csv
Updated dat

In [18]:


def latest_date(file_list):
    date_list = []
    for file_name in file_list:
        date_ = datetime.datetime.strptime(file_name.split('_')[6], '%Y%m%d').date()
        date_list.append(date_)

    max_date = max(date_list)
    print("The latest date is:", max_date)
    return max_date

list_files = [file[10:] for file in glob.glob('zip_files/*.zip')]
last_date = latest_date(list_files)

The latest date is: 2024-08-08


In [16]:

base_dir = "./zip_files"
# # arr = os.listdir(base_dir)

filePresent = set()
for filename in os.listdir(base_dir):
    if filename.endswith('.zip'):
        filePresent.add(filename[22:30])


data = pd.read_excel("./nse_data_url_temp.xlsx")
for link in data['BhavCopy'].to_numpy():
    if link[66:74] not in filePresent:
        print(link[66:74])

20240501
20240502
20240503
20240506
20240507
20240508
20240509
20240510
20240513
20240514
20240515
20240516
20240517
20240520
20240521
20240522
20240523
20240524
20240527
20240528
20240529
20240530
20240531
20240603
20240604
20240605
20240606
20240607
20240610
20240611
20240612
20240613
20240614
20240617
20240618
20240619
20240620
20240621
20240624
20240625
20240626
20240627
20240628
20240701
20240702
20240704
20240705
20240709
20240710
20240711
20240717


In [4]:
existing_parquet_file = 'NSE-Data-2024.parquet'
existing_df = pd.read_parquet(existing_parquet_file)
print(existing_df)

            TradDt       BizDt Sgmt  Src FinInstrmTp  FinInstrmId  \
0       2024-01-01  2024-01-01   CM  NSE         STK        11394   
1       2024-01-01  2024-01-01   CM  NSE         STK           13   
2       2024-01-01  2024-01-01   CM  NSE         STK          350   
3       2024-01-01  2024-01-01   CM  NSE         STK        31475   
4       2024-01-01  2024-01-01   CM  NSE         STK        17764   
...            ...         ...  ...  ...         ...          ...   
347705  2024-07-26  2024-07-26   CM  NSE         STK        13500   
347706  2024-07-26  2024-07-26   CM  NSE         STK        28913   
347707  2024-07-26  2024-07-26   CM  NSE         STK        24366   
347708  2024-07-26  2024-07-26   CM  NSE         STK        23123   
347709  2024-07-26  2024-07-26   CM  NSE         STK         3324   

                ISIN    TckrSymb SctySrs  XpryDt  ...  Rmks  Rsvd01  Rsvd02  \
0       INE358U01012        ZOTA      EQ     NaN  ...   NaN     NaN     NaN   
1       INE11

In [8]:
pd.read_parquet("./kaggle/bhavcopy_dataset.parquet")

,TradDt,BizDt,Sgmt,Src,FinInstrmTp,FinInstrmId,ISIN,TckrSymb,SctySrs,XpryDt,...,Rmks,Rsvd01,Rsvd02,Rsvd03,Rsvd04,Unnamed: 34,Rsvd1,Rsvd2,Rsvd3,Rsvd4
0,2024-01-01,2024-01-01,CM,NSE,STK,11394,INE358U01012,ZOTA,EQ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-01,2024-01-01,CM,NSE,STK,13,INE117A01022,ABB,EQ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-01-01,2024-01-01,CM,NSE,STK,350,INE459A01010,BANARISUG,EQ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-01,2024-01-01,CM,NSE,STK,31475,INE231G01010,ORBTEXP,EQ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-01-01,2024-01-01,CM,NSE,STK,17764,INE056I01017,REFEX,EQ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339411,2024-07-18,2024-07-18,CM,NSE,STK,5012,IN0020210095,610GS2031,GS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339412,2024-07-18,2024-07-18,CM,NSE,STK,2085,INE288A01013,MAHSCOOTER,EQ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339413,2024-07-18,2024-07-18,CM,NSE,STK,9867,INE747B01016,NEXTMEDIA,EQ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339414,2024-07-18,2024-07-18,CM,NSE,STK,4973,INF204KC1089,PHARMABEES,EQ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Upload .parquet file to the kaggle server.

In [5]:
import os
os.getcwd()

'c:\\Users\\lenovo\\OneDrive - IIAIAC.org\\NSE\\NSE_Case_Study'

In [9]:
from kaggle.api.kaggle_api_extended import KaggleApi
import os

# Set the path to the kaggle.json file
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()

# Initialize the Kaggle API
api = KaggleApi()
api.authenticate()

# Define the path to your file and the dataset
file_path = './kaggle/bhavcopy_dataset.parquet'  # Replace with the actual path to your Parquet file
dataset_path = 'vanshitaarya/nse-scripts-data'  # Replace with your Kaggle username and desired dataset name
message = 'Uploading Parquet file'  # Commit message

# Ensure the metadata file is in the same directory as your Parquet file
metadata_file_path = os.path.join(os.path.dirname(file_path), 'kaggle' ,'dataset-metadata.json')

# Check if the dataset already exists
try:
    api.dataset_list_files(dataset_path)
    dataset_exists = True
except:
    dataset_exists = False

if not dataset_exists:
    # Create a new dataset
    api.dataset_create_new(
        folder=os.path.dirname(file_path),
        convert_to_csv=False,
        dir_mode='skip',
        public=True,
        quiet=False
    )
else:
    # Upload a file to the existing dataset
    api.dataset_create_version(
        folder=os.path.dirname(file_path),
        version_notes=message,
        convert_to_csv=False,
        delete_old_versions=False,
        dir_mode='skip',
        quiet=False
    )


Starting upload for file bhavcopy_dataset copy.parquet


100%|██████████| 14.4M/14.4M [01:23<00:00, 181kB/s] 


Upload successful: bhavcopy_dataset copy.parquet (14MB)
Starting upload for file bhavcopy_dataset.parquet


100%|██████████| 14.8M/14.8M [00:46<00:00, 330kB/s] 


Upload successful: bhavcopy_dataset.parquet (15MB)


https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240501_F_0000.csv.zip
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240502_F_0000.csv.zip
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240503_F_0000.csv.zip
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240506_F_0000.csv.zip
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240507_F_0000.csv.zip
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240508_F_0000.csv.zip
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240509_F_0000.csv.zip
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240510_F_0000.csv.zip
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240513_F_0000.csv.zip
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240514_F_0000.csv.zip
https://nsearchives.nseindia.com/content/cm/BhavCopy_NSE_CM_0_0_0_20240515_F_0000.csv.zip
https://ns

In [6]:

import shutil

original = r"original path of the file\file_name.file_extension"
target = r"target path to store the file\file_name.file_extension"

shutil.move(original, target)

3
5
6
7
15
